# Descripción del proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

## Inicialización

In [96]:
#importemos la librerias que vamos a necesitar 
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Carguemos datos 

In [61]:
#carguemos los datos 
megaline = pd.read_csv('./users_behavior.csv')

## Revisemos los datos

In [62]:
megaline.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [63]:
megaline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [64]:
megaline.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Los datos están en el formato correcto 

## Segmentación de los datos

Una vez que los datos están listos, vamos a hacer la segmentación del dt en 3 (Entrenamiento, Validación y Prueba) para poder crear nuestro modelo

In [65]:

#Dividimos nuestro DT en 2 (Entrenamiento y Validación&Prueba)
mega_train , mega_valid_test = train_test_split(megaline, random_state= 345, test_size=0.3)
#Dividamos nuestro nuevo set mega_valid_test en 2 (Validación y Prueba)
mega_valid , mega_test = train_test_split(mega_valid_test, random_state=345, test_size=0.5)

Ya tenemos nuestros 3 datasets: 
- mega_train para entrenar al modelo (70%)
- mega_valid para validar nuestro modelo (15%)
- mega_test para probar nuestro modelo. (15%)

Decidí entrenar el modelo con el 70% de la información porque creo que mientras mas información tenga para entrenar será más exacto. Pero ya lo veremos 

In [66]:
#Separemos los features del target de cada dt 
train_features = mega_train.drop(columns='is_ultra')
train_target = mega_train['is_ultra']
valid_features = mega_valid.drop(columns='is_ultra')
valid_target = mega_valid['is_ultra']
test_features = mega_test.drop(columns='is_ultra')
test_target = mega_test['is_ultra']

Tenemos nuestros datos listos para crear nuestros modelos. Nuestro target es la columna 'is_ultra' y nuestro modelo tendrá que predecir si los usuarios serán Smart(0) o Ultra(1). por lo que utilizaremos modelos de clasificación. 

## Creación y validación de Modelos 

### Decision Tree

In [67]:
#vamos a crear un modelo y probemos varias profundidades para encontrar la correcta mediante un bucle 
for i in range(1,6):
    tree_model = DecisionTreeClassifier(random_state=345, max_depth=i)
    tree_model.fit(train_features,train_target)
    tree_predictions =  tree_model.predict(valid_features)
    print('max_depth=',i, ":",end='')
    print(accuracy_score(valid_target,tree_predictions))

max_depth= 1 :0.7261410788381742
max_depth= 2 :0.7427385892116183
max_depth= 3 :0.7634854771784232
max_depth= 4 :0.7676348547717843
max_depth= 5 :0.7655601659751037


Observamos que de los modelos de árbol, el modelo con una max depth de 4, es el que tiene mayor exactitud en 0.7676348547717843

### Random Forrest 

In [73]:
# Vamos a crear un bucle que pruebe diferentes números de estimadores para determinar cual es el mejor
bst_score = 0
bst_est = 0 
for est in range(1, 101):
    r_model = RandomForestClassifier(random_state=345, n_estimators=est)
    r_model.fit(train_features,train_target)
    score_r = r_model.score(valid_features,valid_target)
    if score_r > bst_score:
        bst_score = score_r
        bst_est = est
print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(bst_est, bst_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 12): 0.7925311203319502


Podemos observar que en el modelo de Random Forrest la mejor predicción la realiza con 12 bosque obteniendo 0.7925311203319502 de exactitud

### Regresión Logística 

In [70]:
log_model = LogisticRegression(random_state=345, solver='liblinear')
log_model.fit(train_features,train_target)

train_score  = log_model.score(train_features,train_target)
log_score = log_model.score(valid_features,valid_target)

print('Exactitud del modelo en entranamiento:',train_score)
print('Exactitud del modelo en validación', log_score)

Exactitud del modelo en entranamiento: 0.7572254335260116
Exactitud del modelo en validación 0.7116182572614108


De los 3 modelos que hemos probado, el modelo de Random Forrest classifier es el que muestra una mayor exactitud al validar con el dataset de validación  CON 0.7925311203319502 DE EXACTITUD y es el modelo hemos decidido probar con el dataset de prueba. 

## Calidad del Modelo 

Ahora probemos el modelo ganador con el dataset de prueba. 

El modelo ganador fue el bosque de decisión con n_estimators = 12

In [93]:
#Recapitulemos con el modelo y el dataset de validación y veamos su exactitud
val_model = RandomForestClassifier(random_state=345, n_estimators=12)
val_model.fit(train_features,train_target)
val_score = val_model.score(valid_features,valid_target)
print('Score de validación:', val_score)

#probemos el modelo con el dataset de prueba y validemos su exactitud
best_model = RandomForestClassifier(random_state=345, n_estimators = 12)
best_model.fit(train_features,train_target)
best_score = best_model.score(test_features,test_target)

print('Model score:', best_score)


Score de validación: 0.7925311203319502
Model score: 0.782608695652174


Después de probar el modelo, observamos que tiene una exactitud de 0.782608695652174 . Lo cual comparado con el umbral de exactitud que tenemos para este proyecto de 0.75 es muy bueno. 

## Conclusiones 

Probamos 3 modelos de clasificación distintos y corrimos pruebas para determinar los mejores parámetros. 
Podemos concluir que de los 3 modelos , el Random Forrest Classifier con un n_estimator de 12 es el que mejor exactitud tiene y por eso hemos decidido usarlo para probarlo con nuestro set de prueba. 
El umbral de exactitud que arrojó nuestra prueba de calidad es de 0.78 ,contra 0.75 que nos pide el proyecto. lo cual supera las expectativas. 

